In [1]:
# QVM Engine v3j - Integrated Strategy with Validated Factors (Full Implementation)

=============================================================================
CONFIGURATION AND DATABASE SETUP
=============================================================================

# QVM Engine v3j - Integrated Strategy with Validated Factors

**Objective:** Full implementation of QVM Engine v3j with statistically validated factors:
- Regime detection
- Value factors (P/E + FCF Yield)
- Quality factors (ROAA + Piotroski F-Score)
- Momentum factors (Multi-horizon + Low-Volatility)
- Integrated portfolio construction

**File:** 08_integrated_strategy_with_validated_factors.py

In [2]:
# Core scientific libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from pathlib import Path
import sys
import yaml

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Database connectivity
from sqlalchemy import create_engine, text

# IMPORTS AND SETUP

In [3]:
# Environment Setup
warnings.filterwarnings('ignore')

# Add Project Root to Python Path
try:
    current_path = Path.cwd()
    while not (current_path / 'production').is_dir():
        if current_path.parent == current_path:
            raise FileNotFoundError("Could not find the 'production' directory.")
        current_path = current_path.parent
    
    project_root = current_path
    
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    
    from production.database.connection import get_database_manager
    from production.database.mappings.financial_mapping_manager import FinancialMappingManager
    print(f"✅ Successfully imported production modules.")
    print(f"   - Project Root set to: {project_root}")

except (ImportError, FileNotFoundError) as e:
    print(f"❌ ERROR: Could not import production modules. Please check your directory structure.")
    print(f"   - Final Path Searched: {project_root}")
    print(f"   - Error: {e}")
    raise

✅ Successfully imported production modules.
   - Project Root set to: /Users/raymond/Documents/Projects/factor-investing-public


# CONFIGURATION AND DATABASE SETUP

In [4]:
QVM_CONFIG = {
    # Backtest Parameters
    "strategy_name": "QVM_Engine_v3j_Validated_Factors",
    "backtest_start_date": "2016-01-01",
    "backtest_end_date": "2025-07-28",
    "rebalance_frequency": "M", # Monthly
    "transaction_cost_bps": 30, # Flat 30bps
    
    # Universe Construction
    "universe": {
        "lookback_days": 63,
        "top_n_stocks": 200,  # Top 200 stocks by ADTV
        "max_position_size": 0.05,
        "max_sector_exposure": 0.30,
        "target_portfolio_size": 20,
    },
    
    # Factor Configuration - Validated Factors Structure
    "factors": {
        "value_weight": 0.33,      # Value factors (P/E + FCF Yield)
        "quality_weight": 0.33,    # Quality factors (ROAA + F-Score)
        "momentum_weight": 0.34,   # Momentum factors (Momentum + Low-Vol)
        
        # Value Factors (0.33 total weight)
        "value_factors": {
            "pe_weight": 0.5,        # 0.165 of total (contrarian - lower is better)
            "fcf_yield_weight": 0.5  # 0.165 of total (positive - higher is better)
        },
        
        # Quality Factors (0.33 total weight)
        "quality_factors": {
            "roaa_weight": 0.5,    # 0.165 of total (positive - higher is better)
            "fscore_weight": 0.5   # 0.165 of total (positive - higher is better)
        },
        
        # Momentum Factors (0.34 total weight)
        "momentum_factors": {
            "momentum_weight": 0.5, # 0.17 of total (mixed signals)
            "low_vol_weight": 0.5   # 0.17 of total (defensive - inverse volatility)
        },
        
        # Factor Calculation Parameters
        "momentum_horizons": [21, 63, 126, 252], # 1M, 3M, 6M, 12M
        "skip_months": 1,
        "fundamental_lag_days": 45,  # 45-day lag for announcement delay
        "volatility_lookback": 252,  # 252-day rolling window for low-vol
        "fcf_imputation_rate": 0.30  # Expected CapEx imputation rate
    },
    
    "regime": {
        "lookback_period": 90,          # 90 days lookback period
        "volatility_threshold": 0.0140, # 1.40% (75th percentile from real data)
        "return_threshold": 0.0012,     # 0.12% (75th percentile from real data)
        "low_return_threshold": 0.0002  # 0.02% (corrected 25th percentile)
    }
}

print("\n⚙️  QVM Engine v3j Validated Factors Configuration Loaded:")
print(f"   - Strategy: {QVM_CONFIG['strategy_name']}")
print(f"   - Period: {QVM_CONFIG['backtest_start_date']} to {QVM_CONFIG['backtest_end_date']}")
print(f"   - Universe: Top {QVM_CONFIG['universe']['top_n_stocks']} stocks by ADTV")
print(f"   - Value Factors: P/E + FCF Yield (33% weight)")
print(f"   - Quality Factors: ROAA + Piotroski F-Score (33% weight)")
print(f"   - Momentum Factors: Multi-horizon + Low-Volatility (34% weight)")
print(f"   - Regime Detection: Fixed thresholds with 4-regime classification")
print(f"   - Performance: Pre-computed data + Vectorized operations")


⚙️  QVM Engine v3j Validated Factors Configuration Loaded:
   - Strategy: QVM_Engine_v3j_Validated_Factors
   - Period: 2016-01-01 to 2025-07-28
   - Universe: Top 200 stocks by ADTV
   - Value Factors: P/E + FCF Yield (33% weight)
   - Quality Factors: ROAA + Piotroski F-Score (33% weight)
   - Momentum Factors: Multi-horizon + Low-Volatility (34% weight)
   - Regime Detection: Fixed thresholds with 4-regime classification
   - Performance: Pre-computed data + Vectorized operations


# DATABASE CONNECTION

In [5]:
def create_db_connection():
    """Establishes a SQLAlchemy database engine connection."""
    try:
        db_manager = get_database_manager()
        engine = db_manager.get_engine()
        
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        print(f"\n✅ Database connection established successfully.")
        return engine

    except Exception as e:
        print(f"❌ FAILED to connect to the database.")
        print(f"   - Error: {e}")
        return None

# Create the engine for this session
engine = create_db_connection()

if engine is None:
    raise ConnectionError("Database connection failed. Halting execution.")

2025-08-04 01:45:16,612 - production.database.connection - INFO - Database configuration loaded from /Users/raymond/Documents/Projects/factor-investing-public/config/database.yml
2025-08-04 01:45:16,613 - production.database.connection - INFO - DatabaseManager initialized for environment: production
2025-08-04 01:45:16,878 - production.database.connection - INFO - SQLAlchemy engine created successfully



✅ Database connection established successfully.


# VALIDATED FACTORS CALCULATION CLASSES

In [6]:
class ValidatedFactorsCalculator:
    """
    Calculator for the three statistically validated factors:
    1. Low-Volatility Factor (defensive momentum)
    2. Piotroski F-Score Factor (quality assessment)
    3. FCF Yield Factor (value enhancement)
    """
    
    def __init__(self, engine):
        self.engine = engine
        print("✅ ValidatedFactorsCalculator initialized")
    
    def calculate_low_volatility_factor(self, price_data: pd.DataFrame, lookback_days: int = 252) -> pd.Series:
        """
        Calculate Low-Volatility factor using inverse 252-day rolling volatility.
        
        Args:
            price_data: DataFrame with 'ticker', 'date', 'close' columns
            lookback_days: Rolling window for volatility calculation (default: 252)
        
        Returns:
            Series with low-volatility scores (inverse relationship)
        """
        try:
            # Pivot data for vectorized calculation
            price_pivot = price_data.pivot(index='date', columns='ticker', values='close')
            
            # Calculate rolling volatility
            volatility = price_pivot.rolling(lookback_days).std() * np.sqrt(252)
            
            # Apply inverse relationship (lower volatility = higher score)
            low_vol_score = 1 / volatility
            
            # Stack back to long format
            low_vol_stacked = low_vol_score.stack().reset_index()
            low_vol_stacked.columns = ['date', 'ticker', 'low_vol_score']
            
            # Remove infinite values and outliers
            low_vol_stacked = low_vol_stacked.replace([np.inf, -np.inf], np.nan)
            low_vol_stacked = low_vol_stacked.dropna()
            
            # Winsorize outliers (top and bottom 1%)
            q_low = low_vol_stacked['low_vol_score'].quantile(0.01)
            q_high = low_vol_stacked['low_vol_score'].quantile(0.99)
            low_vol_stacked['low_vol_score'] = low_vol_stacked['low_vol_score'].clip(q_low, q_high)
            
            print(f"   ✅ Low-Volatility factor calculated: {len(low_vol_stacked):,} observations")
            return low_vol_stacked
            
        except Exception as e:
            print(f"   ❌ Error calculating Low-Volatility factor: {e}")
            return pd.DataFrame()
    
    def calculate_piotroski_fscore(self, tickers: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """
        Calculate Piotroski F-Score with sector-specific implementations.
        
        Args:
            tickers: List of tickers to analyze
            analysis_date: Date for analysis
        
        Returns:
            DataFrame with F-Scores by sector
        """
        try:
            # Get sector information
            sector_query = text("""
                SELECT ticker, sector
                FROM master_info
                WHERE ticker IN :tickers
            """)
            
            ticker_list = tuple(tickers)
            sector_df = pd.read_sql(sector_query, self.engine, params={'tickers': ticker_list})
            
            if sector_df.empty:
                print("   ⚠️  No sector data found")
                return pd.DataFrame()
            
            # Group by sector and calculate F-Scores
            fscore_results = []
            
            for sector in sector_df['sector'].unique():
                sector_tickers = sector_df[sector_df['sector'] == sector]['ticker'].tolist()
                
                if sector == 'Banking':
                    sector_fscores = self._calculate_banking_fscore(sector_tickers, analysis_date)
                elif sector == 'Securities':
                    sector_fscores = self._calculate_securities_fscore(sector_tickers, analysis_date)
                else:
                    sector_fscores = self._calculate_nonfin_fscore(sector_tickers, analysis_date)
                
                if not sector_fscores.empty:
                    sector_fscores['sector'] = sector
                    fscore_results.append(sector_fscores)
            
            if fscore_results:
                combined_fscores = pd.concat(fscore_results, ignore_index=True)
                print(f"   ✅ Piotroski F-Score calculated: {len(combined_fscores):,} observations across {len(combined_fscores['sector'].unique())} sectors")
                return combined_fscores
            else:
                print("   ⚠️  No F-Score data calculated")
                return pd.DataFrame()
                
        except Exception as e:
            print(f"   ❌ Error calculating Piotroski F-Score: {e}")
            return pd.DataFrame()
    
    def _calculate_nonfin_fscore(self, tickers: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """
        Calculate F-Score for non-financial companies (9 tests).
        
        Tests:
        1. ROA > 0 (NetProfit_TTM / AvgTotalAssets)
        2. CFO > 0 (NetCFO_TTM > 0)
        3. ΔROA > 0 (ROA improvement)
        4. Accruals < CFO (quality of earnings)
        5. ΔLeverage < 0 (decreasing leverage)
        6. ΔCurrent Ratio > 0 (improving liquidity)
        7. No new shares issued
        8. ΔGross Margin > 0 (improving profitability)
        9. ΔAsset Turnover > 0 (improving efficiency)
        """
        try:
            # Get data from intermediary_calculations_enhanced
            query = text("""
                SELECT 
                    ticker,
                    year,
                    quarter,
                    NetProfit_TTM,
                    NetCFO_TTM,
                    AvgTotalAssets,
                    TotalDebt,
                    CurrentAssets,
                    CurrentLiabilities,
                    TotalEquity,
                    Revenue_TTM,
                    GrossProfit_TTM,
                    SharesOutstanding
                FROM intermediary_calculations_enhanced
                WHERE ticker IN :tickers
                AND year >= YEAR(:analysis_date) - 2
                ORDER BY ticker, year, quarter
            """)
            
            data = pd.read_sql(query, self.engine, 
                             params={'tickers': tuple(tickers), 'analysis_date': analysis_date})
            
            if data.empty:
                return pd.DataFrame()
            
            # Calculate metrics and tests
            results = []
            
            for ticker in data['ticker'].unique():
                ticker_data = data[data['ticker'] == ticker].sort_values(['year', 'quarter'])
                
                if len(ticker_data) < 2:  # Need at least 2 periods for changes
                    continue
                
                # Get current and previous period
                current = ticker_data.iloc[-1]
                previous = ticker_data.iloc[-2]
                
                # Test 1: ROA > 0
                roa_current = current['NetProfit_TTM'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                test1 = 1 if roa_current > 0 else 0
                
                # Test 2: CFO > 0
                test2 = 1 if current['NetCFO_TTM'] > 0 else 0
                
                # Test 3: ΔROA > 0
                roa_previous = previous['NetProfit_TTM'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test3 = 1 if roa_current > roa_previous else 0
                
                # Test 4: Accruals < CFO (simplified: NetProfit < CFO)
                test4 = 1 if current['NetProfit_TTM'] < current['NetCFO_TTM'] else 0
                
                # Test 5: ΔLeverage < 0 (decreasing debt/equity)
                leverage_current = current['TotalDebt'] / current['TotalEquity'] if current['TotalEquity'] > 0 else 0
                leverage_previous = previous['TotalDebt'] / previous['TotalEquity'] if previous['TotalEquity'] > 0 else 0
                test5 = 1 if leverage_current < leverage_previous else 0
                
                # Test 6: ΔCurrent Ratio > 0
                cr_current = current['CurrentAssets'] / current['CurrentLiabilities'] if current['CurrentLiabilities'] > 0 else 0
                cr_previous = previous['CurrentAssets'] / previous['CurrentLiabilities'] if previous['CurrentLiabilities'] > 0 else 0
                test6 = 1 if cr_current > cr_previous else 0
                
                # Test 7: No new shares issued (simplified: shares unchanged or decreased)
                test7 = 1 if current['SharesOutstanding'] <= previous['SharesOutstanding'] else 0
                
                # Test 8: ΔGross Margin > 0
                gm_current = current['GrossProfit_TTM'] / current['Revenue_TTM'] if current['Revenue_TTM'] > 0 else 0
                gm_previous = previous['GrossProfit_TTM'] / previous['Revenue_TTM'] if previous['Revenue_TTM'] > 0 else 0
                test8 = 1 if gm_current > gm_previous else 0
                
                # Test 9: ΔAsset Turnover > 0
                at_current = current['Revenue_TTM'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                at_previous = previous['Revenue_TTM'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test9 = 1 if at_current > at_previous else 0
                
                # Calculate total F-Score
                fscore = test1 + test2 + test3 + test4 + test5 + test6 + test7 + test8 + test9
                
                results.append({
                    'ticker': ticker,
                    'fscore': fscore,
                    'test1_roa': test1,
                    'test2_cfo': test2,
                    'test3_delta_roa': test3,
                    'test4_accruals': test4,
                    'test5_delta_leverage': test5,
                    'test6_delta_current_ratio': test6,
                    'test7_no_new_shares': test7,
                    'test8_delta_gross_margin': test8,
                    'test9_delta_asset_turnover': test9
                })
            
            return pd.DataFrame(results)
            
        except Exception as e:
            print(f"   ❌ Error calculating non-financial F-Score: {e}")
            return pd.DataFrame()
    
    def _calculate_banking_fscore(self, tickers: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """
        Calculate F-Score for banking companies (9 tests).
        
        Tests:
        1. NIM > 0 (Net Interest Margin)
        2. ROA > 0 (NetProfit_TTM / AvgTotalAssets)
        3. ΔROA > 0 (ROA improvement)
        4. ΔNIM > 0 (NIM improvement)
        5. ΔEfficiency Ratio < 0 (improving efficiency)
        6. ΔCapital Adequacy > 0 (improving capital)
        7. No new shares issued
        8. ΔRevenue Growth > 0
        9. ΔAsset Quality > 0
        """
        try:
            # Get data from intermediary_calculations_banking
            query = text("""
                SELECT 
                    ticker,
                    year,
                    quarter,
                    NetProfit_TTM,
                    AvgTotalAssets,
                    NetInterestIncome_TTM,
                    AvgInterestEarningAssets,
                    OperatingExpenses_TTM,
                    Revenue_TTM,
                    TotalEquity,
                    NonPerformingLoans,
                    TotalLoans,
                    SharesOutstanding
                FROM intermediary_calculations_banking
                WHERE ticker IN :tickers
                AND year >= YEAR(:analysis_date) - 2
                ORDER BY ticker, year, quarter
            """)
            
            data = pd.read_sql(query, self.engine, 
                             params={'tickers': tuple(tickers), 'analysis_date': analysis_date})
            
            if data.empty:
                return pd.DataFrame()
            
            # Calculate metrics and tests
            results = []
            
            for ticker in data['ticker'].unique():
                ticker_data = data[data['ticker'] == ticker].sort_values(['year', 'quarter'])
                
                if len(ticker_data) < 2:  # Need at least 2 periods for changes
                    continue
                
                # Get current and previous period
                current = ticker_data.iloc[-1]
                previous = ticker_data.iloc[-2]
                
                # Test 1: NIM > 0
                nim_current = current['NetInterestIncome_TTM'] / current['AvgInterestEarningAssets'] if current['AvgInterestEarningAssets'] > 0 else 0
                test1 = 1 if nim_current > 0 else 0
                
                # Test 2: ROA > 0
                roa_current = current['NetProfit_TTM'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                test2 = 1 if roa_current > 0 else 0
                
                # Test 3: ΔROA > 0
                roa_previous = previous['NetProfit_TTM'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test3 = 1 if roa_current > roa_previous else 0
                
                # Test 4: ΔNIM > 0
                nim_previous = previous['NetInterestIncome_TTM'] / previous['AvgInterestEarningAssets'] if previous['AvgInterestEarningAssets'] > 0 else 0
                test4 = 1 if nim_current > nim_previous else 0
                
                # Test 5: ΔEfficiency Ratio < 0 (improving efficiency)
                eff_current = current['OperatingExpenses_TTM'] / current['Revenue_TTM'] if current['Revenue_TTM'] > 0 else 0
                eff_previous = previous['OperatingExpenses_TTM'] / previous['Revenue_TTM'] if previous['Revenue_TTM'] > 0 else 0
                test5 = 1 if eff_current < eff_previous else 0
                
                # Test 6: ΔCapital Adequacy > 0 (improving capital ratio)
                cap_current = current['TotalEquity'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                cap_previous = previous['TotalEquity'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test6 = 1 if cap_current > cap_previous else 0
                
                # Test 7: No new shares issued
                test7 = 1 if current['SharesOutstanding'] <= previous['SharesOutstanding'] else 0
                
                # Test 8: ΔRevenue Growth > 0
                test8 = 1 if current['Revenue_TTM'] > previous['Revenue_TTM'] else 0
                
                # Test 9: ΔAsset Quality > 0 (decreasing NPL ratio)
                npl_current = current['NonPerformingLoans'] / current['TotalLoans'] if current['TotalLoans'] > 0 else 0
                npl_previous = previous['NonPerformingLoans'] / previous['TotalLoans'] if previous['TotalLoans'] > 0 else 0
                test9 = 1 if npl_current < npl_previous else 0
                
                # Calculate total F-Score
                fscore = test1 + test2 + test3 + test4 + test5 + test6 + test7 + test8 + test9
                
                results.append({
                    'ticker': ticker,
                    'fscore': fscore,
                    'test1_nim': test1,
                    'test2_roa': test2,
                    'test3_delta_roa': test3,
                    'test4_delta_nim': test4,
                    'test5_delta_efficiency': test5,
                    'test6_delta_capital': test6,
                    'test7_no_new_shares': test7,
                    'test8_revenue_growth': test8,
                    'test9_asset_quality': test9
                })
            
            return pd.DataFrame(results)
            
        except Exception as e:
            print(f"   ❌ Error calculating banking F-Score: {e}")
            return pd.DataFrame()
    
    def _calculate_securities_fscore(self, tickers: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """
        Calculate F-Score for securities companies (9 tests).
        
        Tests:
        1. Trading Income > 0 (NetTradingIncome_TTM)
        2. Brokerage Revenue > 0 (BrokerageRevenue_TTM)
        3. ΔTrading Income > 0
        4. ΔBrokerage Revenue > 0
        5. ΔEfficiency Ratio < 0 (improving efficiency)
        6. ΔCapital Adequacy > 0 (improving capital)
        7. No new shares issued
        8. ΔRevenue Growth > 0
        9. ΔAsset Quality > 0
        """
        try:
            # Get data from intermediary_calculations_securities
            query = text("""
                SELECT 
                    ticker,
                    year,
                    quarter,
                    NetTradingIncome_TTM,
                    BrokerageRevenue_TTM,
                    OperatingExpenses_TTM,
                    Revenue_TTM,
                    TotalEquity,
                    AvgTotalAssets,
                    SharesOutstanding
                FROM intermediary_calculations_securities
                WHERE ticker IN :tickers
                AND year >= YEAR(:analysis_date) - 2
                ORDER BY ticker, year, quarter
            """)
            
            data = pd.read_sql(query, self.engine, 
                             params={'tickers': tuple(tickers), 'analysis_date': analysis_date})
            
            if data.empty:
                return pd.DataFrame()
            
            # Calculate metrics and tests
            results = []
            
            for ticker in data['ticker'].unique():
                ticker_data = data[data['ticker'] == ticker].sort_values(['year', 'quarter'])
                
                if len(ticker_data) < 2:  # Need at least 2 periods for changes
                    continue
                
                # Get current and previous period
                current = ticker_data.iloc[-1]
                previous = ticker_data.iloc[-2]
                
                # Test 1: Trading Income > 0
                test1 = 1 if current['NetTradingIncome_TTM'] > 0 else 0
                
                # Test 2: Brokerage Revenue > 0
                test2 = 1 if current['BrokerageRevenue_TTM'] > 0 else 0
                
                # Test 3: ΔTrading Income > 0
                test3 = 1 if current['NetTradingIncome_TTM'] > previous['NetTradingIncome_TTM'] else 0
                
                # Test 4: ΔBrokerage Revenue > 0
                test4 = 1 if current['BrokerageRevenue_TTM'] > previous['BrokerageRevenue_TTM'] else 0
                
                # Test 5: ΔEfficiency Ratio < 0 (improving efficiency)
                eff_current = current['OperatingExpenses_TTM'] / current['Revenue_TTM'] if current['Revenue_TTM'] > 0 else 0
                eff_previous = previous['OperatingExpenses_TTM'] / previous['Revenue_TTM'] if previous['Revenue_TTM'] > 0 else 0
                test5 = 1 if eff_current < eff_previous else 0
                
                # Test 6: ΔCapital Adequacy > 0 (improving capital ratio)
                cap_current = current['TotalEquity'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                cap_previous = previous['TotalEquity'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test6 = 1 if cap_current > cap_previous else 0
                
                # Test 7: No new shares issued
                test7 = 1 if current['SharesOutstanding'] <= previous['SharesOutstanding'] else 0
                
                # Test 8: ΔRevenue Growth > 0
                test8 = 1 if current['Revenue_TTM'] > previous['Revenue_TTM'] else 0
                
                # Test 9: ΔAsset Quality > 0 (simplified: improving ROA)
                roa_current = current['NetTradingIncome_TTM'] / current['AvgTotalAssets'] if current['AvgTotalAssets'] > 0 else 0
                roa_previous = previous['NetTradingIncome_TTM'] / previous['AvgTotalAssets'] if previous['AvgTotalAssets'] > 0 else 0
                test9 = 1 if roa_current > roa_previous else 0
                
                # Calculate total F-Score
                fscore = test1 + test2 + test3 + test4 + test5 + test6 + test7 + test8 + test9
                
                results.append({
                    'ticker': ticker,
                    'fscore': fscore,
                    'test1_trading_income': test1,
                    'test2_brokerage_revenue': test2,
                    'test3_delta_trading': test3,
                    'test4_delta_brokerage': test4,
                    'test5_delta_efficiency': test5,
                    'test6_delta_capital': test6,
                    'test7_no_new_shares': test7,
                    'test8_revenue_growth': test8,
                    'test9_asset_quality': test9
                })
            
            return pd.DataFrame(results)
            
        except Exception as e:
            print(f"   ❌ Error calculating securities F-Score: {e}")
            return pd.DataFrame()
    
    def calculate_fcf_yield(self, tickers: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """
        Calculate FCF Yield factor with imputation handling.
        
        Args:
            tickers: List of tickers to analyze
            analysis_date: Date for analysis
        
        Returns:
            DataFrame with FCF Yield scores
        """
        try:
            # Get fundamental data for FCF calculation
            fundamental_query = text("""
                WITH fundamental_data AS (
                    SELECT 
                        fv.ticker,
                        fv.year,
                        fv.quarter,
                        fv.item_id,
                        fv.statement_type,
                        SUM(fv.value / 1e9) as value_bn
                    FROM fundamental_values fv
                    WHERE fv.ticker IN :tickers
                    AND fv.item_id IN (1, 2, 3)  -- NetProfit, TotalAssets, CapEx
                    AND fv.year >= YEAR(:analysis_date) - 2
                    GROUP BY fv.ticker, fv.year, fv.quarter, fv.item_id, fv.statement_type
                ),
                netprofit_ttm AS (
                    SELECT ticker, year, quarter, value_bn as netprofit_ttm
                    FROM fundamental_data
                    WHERE item_id = 1 AND statement_type = 'PL'
                ),
                totalassets_ttm AS (
                    SELECT ticker, year, quarter, value_bn as totalassets_ttm
                    FROM fundamental_data
                    WHERE item_id = 2 AND statement_type = 'BS'
                ),
                capex_ttm AS (
                    SELECT ticker, year, quarter, value_bn as capex_ttm
                    FROM fundamental_data
                    WHERE item_id = 3 AND statement_type = 'CF'
                )
                SELECT 
                    np.ticker,
                    np.year,
                    np.quarter,
                    np.netprofit_ttm,
                    ta.totalassets_ttm,
                    cx.capex_ttm
                FROM netprofit_ttm np
                LEFT JOIN totalassets_ttm ta ON np.ticker = ta.ticker AND np.year = ta.year AND np.quarter = ta.quarter
                LEFT JOIN capex_ttm cx ON np.ticker = cx.ticker AND np.year = cx.year AND np.quarter = cx.quarter
                WHERE np.netprofit_ttm > 0
                AND ta.totalassets_ttm > 0
            """)
            
            fundamental_data = pd.read_sql(fundamental_query, self.engine,
                                         params={'tickers': tuple(tickers), 'analysis_date': analysis_date})
            
            if fundamental_data.empty:
                print("   ⚠️  No fundamental data found for FCF calculation")
                return pd.DataFrame()
            
            # Get market cap data
            market_cap_query = text("""
                SELECT ticker, market_cap
                FROM vcsc_daily_data_complete
                WHERE ticker IN :tickers
                AND trading_date = :analysis_date
            """)
            
            market_cap_data = pd.read_sql(market_cap_query, self.engine,
                                        params={'tickers': tuple(tickers), 'analysis_date': analysis_date})
            
            if market_cap_data.empty:
                print("   ⚠️  No market cap data found")
                return pd.DataFrame()
            
            # Calculate FCF and FCF Yield
            fundamental_data = fundamental_data.merge(market_cap_data, on='ticker', how='inner')
            
            # Impute missing CapEx (conservative estimate: -5% of NetCFO)
            imputation_rate = 0.0
            if 'capex_ttm' in fundamental_data.columns:
                missing_capex = fundamental_data['capex_ttm'].isna().sum()
                total_obs = len(fundamental_data)
                imputation_rate = missing_capex / total_obs if total_obs > 0 else 0
                
                # Impute with conservative estimate
                fundamental_data['capex_ttm'] = fundamental_data['capex_ttm'].fillna(
                    -0.05 * fundamental_data['netprofit_ttm']
                )
            
            # Calculate FCF (simplified: NetProfit - CapEx)
            fundamental_data['fcf'] = fundamental_data['netprofit_ttm'] - fundamental_data['capex_ttm']
            
            # Calculate FCF Yield
            fundamental_data['fcf_yield'] = fundamental_data['fcf'] / fundamental_data['market_cap']
            
            # Clean and filter
            fcf_results = fundamental_data[['ticker', 'fcf', 'fcf_yield']].copy()
            fcf_results = fcf_results.dropna()
            fcf_results = fcf_results[fcf_results['fcf_yield'] > 0]  # Positive FCF Yield only
            
            # Winsorize outliers
            q_low = fcf_results['fcf_yield'].quantile(0.01)
            q_high = fcf_results['fcf_yield'].quantile(0.99)
            fcf_results['fcf_yield'] = fcf_results['fcf_yield'].clip(q_low, q_high)
            
            print(f"   ✅ FCF Yield calculated: {len(fcf_results):,} observations (imputation rate: {imputation_rate:.2%})")
            return fcf_results
            
        except Exception as e:
            print(f"   ❌ Error calculating FCF Yield: {e}")
            return pd.DataFrame()

class SectorAwareFactorCalculator:
    """
    Sector-aware factor calculator with quality-adjusted P/E and validated factors integration.
    """
    def __init__(self, engine):
        self.engine = engine
        self.validated_calculator = ValidatedFactorsCalculator(engine)
    
    def calculate_sector_aware_pe(self, data: pd.DataFrame) -> pd.DataFrame:
        """Calculate quality-adjusted P/E by sector."""
        if 'roaa' not in data.columns or 'sector' not in data.columns:
            return data
        
        # Create ROAA quintiles within each sector
        def safe_qcut(x):
            try:
                if len(x) < 5:
                    return pd.Series(['Q3'] * len(x), index=x.index)
                return pd.qcut(x, 5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'], duplicates='drop')
            except ValueError:
                return pd.Series(['Q3'] * len(x), index=x.index)
        
        data['roaa_quintile'] = data.groupby('sector')['roaa'].transform(safe_qcut)
        
        # Fill missing quintiles with Q3
        data['roaa_quintile'] = data['roaa_quintile'].fillna('Q3')
        
        # Quality-adjusted P/E weights (higher quality = higher weight)
        quality_weights = {
            'Q1': 0.5,  # Low quality
            'Q2': 0.7,
            'Q3': 1.0,  # Medium quality
            'Q4': 1.3,
            'Q5': 1.5   # High quality
        }
        
        data['quality_adjusted_pe'] = data['roaa_quintile'].map(quality_weights)
        return data
    
    def calculate_momentum_score(self, data: pd.DataFrame) -> pd.DataFrame:
        """Calculate multi-horizon momentum score with correct signal directions."""
        momentum_columns = [col for col in data.columns if col.startswith('momentum_')]
        
        if not momentum_columns:
            return data
        
        # Apply correct signal directions:
        # - 3M and 6M: Positive signals (higher is better)
        # - 1M and 12M: Contrarian signals (lower is better)
        momentum_score = 0.0
        
        for col in momentum_columns:
            if 'momentum_63d' in col or 'momentum_126d' in col:  # 3M and 6M - positive
                momentum_score += data[col]
            elif 'momentum_21d' in col or 'momentum_252d' in col:  # 1M and 12M - contrarian
                momentum_score -= data[col]  # Negative for contrarian
        
        # Equal weight the components
        data['momentum_score'] = momentum_score / len(momentum_columns)
        return data 

# CORE CLASSES AND ENGINE

In [7]:
class RegimeDetector:
    """
    Simple regime detection based on volatility and return thresholds.
    FIXED: Now properly accepts and uses threshold parameters.
    """
    def __init__(self, lookback_period: int = 90, volatility_threshold: float = 0.0140, 
                 return_threshold: float = 0.0012, low_return_threshold: float = 0.0002):
        self.lookback_period = lookback_period
        self.volatility_threshold = volatility_threshold
        self.return_threshold = return_threshold
        self.low_return_threshold = low_return_threshold
        print(f"✅ RegimeDetector initialized with thresholds:")
        print(f"   - Volatility: {self.volatility_threshold:.2%}")
        print(f"   - Return: {self.return_threshold:.2%}")
        print(f"   - Low Return: {self.low_return_threshold:.2%}")
    
    def detect_regime(self, price_data: pd.DataFrame) -> str:
        """Detect market regime based on volatility and return."""
        # Use available data, but require at least 60 days
        min_required_days = 60
        if len(price_data) < min_required_days:
            print(f"   ⚠️  Insufficient data: {len(price_data)} < {min_required_days}")
            return 'Sideways'
        
        # Use all available data up to lookback_period
        available_days = min(len(price_data), self.lookback_period)
        recent_data = price_data.tail(available_days)
        returns = recent_data['close'].pct_change().dropna()
        
        volatility = returns.std()
        avg_return = returns.mean()
        
        # Debug output
        print(f"   🔍 Regime Debug: Vol={volatility:.4f} ({volatility:.2%}), AvgRet={avg_return:.4f} ({avg_return:.2%})")
        print(f"   🔍 Thresholds: VolThresh={self.volatility_threshold:.4f}, RetThresh={self.return_threshold:.4f}, LowRetThresh={self.low_return_threshold:.4f}")
        
        if volatility > self.volatility_threshold:
            if avg_return > self.return_threshold:
                print(f"   📈 Detected: Bull (Vol={volatility:.2%} > {self.volatility_threshold:.2%}, Ret={avg_return:.2%} > {self.return_threshold:.2%})")
                return 'Bull'
            else:
                print(f"   📉 Detected: Bear (Vol={volatility:.2%} > {self.volatility_threshold:.2%}, Ret={avg_return:.2%} <= {self.return_threshold:.2%})")
                return 'Bear'
        else:
            if abs(avg_return) < self.low_return_threshold:
                print(f"   ↔️  Detected: Sideways (Vol={volatility:.2%} <= {self.volatility_threshold:.2%}, |Ret|={abs(avg_return):.2%} < {self.low_return_threshold:.2%})")
                return 'Sideways'
            else:
                print(f"   ⚠️  Detected: Stress (Vol={volatility:.2%} <= {self.volatility_threshold:.2%}, |Ret|={abs(avg_return):.2%} >= {self.low_return_threshold:.2%})")
                return 'Stress'
    
    def get_regime_allocation(self, regime: str) -> float:
        """Get target allocation based on regime."""
        regime_allocations = {
            'Bull': 1.0,      # Fully invested
            'Bear': 0.8,      # 80% invested
            'Sideways': 0.6,  # 60% invested
            'Stress': 0.4     # 40% invested
        }
        return regime_allocations.get(regime, 0.6)

## QVM ENGINE V3J WITH VALIDATED FACTORS

class QVMEngineV3jValidatedFactors:
    """
    QVM Engine v3j with Validated Factors (All Components).
    Uses pre-computed data and vectorized operations for dramatically faster rebalancing.
    Implements the three statistically validated factors:
    - Value factors (P/E + FCF Yield)
    - Quality factors (ROAA + Piotroski F-Score)
    - Momentum factors (Multi-horizon + Low-Volatility)
    """
    def __init__(self, config: dict, price_data: pd.DataFrame, fundamental_data: pd.DataFrame,
                 returns_matrix: pd.DataFrame, benchmark_returns: pd.Series, db_engine, precomputed_data: dict):
        
        self.config = config
        self.engine = db_engine
        self.precomputed_data = precomputed_data
        
        # Slice data to the exact backtest window
        start = pd.Timestamp(config['backtest_start_date'])
        end = pd.Timestamp(config['backtest_end_date'])
        
        self.price_data_raw = price_data[price_data['date'].between(start, end)].copy()
        self.fundamental_data_raw = fundamental_data[fundamental_data['date'].between(start, end)].copy()
        self.daily_returns_matrix = returns_matrix.loc[start:end].copy()
        self.benchmark_returns = benchmark_returns.loc[start:end].copy()
        
        # Initialize components
        self.regime_detector = RegimeDetector(
            lookback_period=config['regime']['lookback_period'],
            volatility_threshold=config['regime']['volatility_threshold'],
            return_threshold=config['regime']['return_threshold'],
            low_return_threshold=config['regime']['low_return_threshold']
        )
        self.sector_calculator = SectorAwareFactorCalculator(db_engine)
        self.validated_calculator = ValidatedFactorsCalculator(db_engine)
        self.mapping_manager = FinancialMappingManager()
        
        # Pre-process precomputed data for faster access
        self._setup_precomputed_data()
        
        print("✅ QVMEngineV3jValidatedFactors initialized.")
        print(f"   - Strategy: {config['strategy_name']}")
        print(f"   - Period: {self.daily_returns_matrix.index.min().date()} to {self.daily_returns_matrix.index.max().date()}")
        print(f"   - Value Factors: P/E + FCF Yield (33% weight)")
        print(f"   - Quality Factors: ROAA + Piotroski F-Score (33% weight)")
        print(f"   - Momentum Factors: Multi-horizon + Low-Volatility (34% weight)")
        print(f"   - Performance: Pre-computed data + Vectorized operations")

    def _setup_precomputed_data(self):
        """Setup precomputed data for fast access during rebalancing."""
        # Create fast lookup structures
        self.universe_lookup = self.precomputed_data['universe'].set_index(['trading_date', 'ticker']).index
        self.fundamental_lookup = self.precomputed_data['fundamentals'].set_index(['date', 'ticker'])
        self.momentum_lookup = self.precomputed_data['momentum'].set_index(['trading_date', 'ticker'])
        
        print("   ✅ Pre-computed data indexed for fast access")

    def run_backtest(self) -> (pd.Series, pd.DataFrame):
        """Executes the full backtesting pipeline with optimized performance."""
        print("\n🚀 Starting QVM Engine v3j validated factors backtest execution...")
        
        rebalance_dates = self._generate_rebalance_dates()
        daily_holdings, diagnostics = self._run_optimized_backtesting_loop(rebalance_dates)
        net_returns = self._calculate_net_returns(daily_holdings)
        
        print("✅ QVM Engine v3j validated factors backtest execution complete.")
        return net_returns, diagnostics

    def _generate_rebalance_dates(self) -> list:
        """Generates monthly rebalance dates based on actual trading days."""
        all_trading_dates = self.daily_returns_matrix.index
        rebal_dates_calendar = pd.date_range(
            start=self.config['backtest_start_date'],
            end=self.config['backtest_end_date'],
            freq=self.config['rebalance_frequency']
        )
        actual_rebal_dates = [all_trading_dates[all_trading_dates.searchsorted(d, side='left')-1] for d in rebal_dates_calendar if d >= all_trading_dates.min()]
        print(f"   - Generated {len(actual_rebal_dates)} monthly rebalance dates.")
        return sorted(list(set(actual_rebal_dates)))

    def _run_optimized_backtesting_loop(self, rebalance_dates: list) -> (pd.DataFrame, pd.DataFrame):
        """Optimized backtesting loop using pre-computed data and validated factors."""
        daily_holdings = pd.DataFrame(0.0, index=self.daily_returns_matrix.index, columns=self.daily_returns_matrix.columns)
        diagnostics_log = []
        
        for i, rebal_date in enumerate(rebalance_dates):
            print(f"   - Processing rebalance {i+1}/{len(rebalance_dates)}: {rebal_date.date()}...", end="")
            
            # Fast universe lookup (no database query)
            universe = self._get_universe_from_precomputed(rebal_date)
            if len(universe) < 5:
                print(" ⚠️ Universe too small. Skipping.")
                continue
            
            # Detect regime
            regime = self._detect_current_regime(rebal_date)
            regime_allocation = self.regime_detector.get_regime_allocation(regime)
            
            # Fast factor calculation with validated factors (no database queries)
            factors_df = self._get_validated_factors_from_precomputed(universe, rebal_date)
            if factors_df.empty:
                print(" ⚠️ No factor data. Skipping.")
                continue
            
            # Apply entry criteria
            qualified_df = self._apply_entry_criteria(factors_df)
            if qualified_df.empty:
                print(" ⚠️ No qualified stocks. Skipping.")
                continue
            
            # Construct portfolio
            target_portfolio = self._construct_portfolio(qualified_df, regime_allocation)
            if target_portfolio.empty:
                print(" ⚠️ Portfolio empty. Skipping.")
                continue
            
            # Apply holdings
            start_period = rebal_date + pd.Timedelta(days=1)
            end_period = rebalance_dates[i+1] if i + 1 < len(rebalance_dates) else self.daily_returns_matrix.index.max()
            holding_dates = self.daily_returns_matrix.index[(self.daily_returns_matrix.index >= start_period) & (self.daily_returns_matrix.index <= end_period)]
            
            daily_holdings.loc[holding_dates] = 0.0
            valid_tickers = target_portfolio.index.intersection(daily_holdings.columns)
            daily_holdings.loc[holding_dates, valid_tickers] = target_portfolio[valid_tickers].values
            
            # Calculate turnover
            if i > 0:
                try:
                    prev_holdings_idx = self.daily_returns_matrix.index.get_loc(rebal_date) - 1
                except KeyError:
                    prev_dates = self.daily_returns_matrix.index[self.daily_returns_matrix.index < rebal_date]
                    if len(prev_dates) > 0:
                        prev_holdings_idx = self.daily_returns_matrix.index.get_loc(prev_dates[-1])
                    else:
                        prev_holdings_idx = -1
                
                prev_holdings = daily_holdings.iloc[prev_holdings_idx] if prev_holdings_idx >= 0 else pd.Series(dtype='float64')
            else:
                prev_holdings = pd.Series(dtype='float64')

            turnover = (target_portfolio - prev_holdings.reindex(target_portfolio.index).fillna(0)).abs().sum() / 2.0
            
            diagnostics_log.append({
                'date': rebal_date,
                'universe_size': len(universe),
                'portfolio_size': len(target_portfolio),
                'regime': regime,
                'regime_allocation': regime_allocation,
                'turnover': turnover
            })
            print(f" ✅ Universe: {len(universe)}, Portfolio: {len(target_portfolio)}, Regime: {regime}, Turnover: {turnover:.2%}")

        if diagnostics_log:
            return daily_holdings, pd.DataFrame(diagnostics_log).set_index('date')
        else:
            return daily_holdings, pd.DataFrame()

    def _get_universe_from_precomputed(self, analysis_date: pd.Timestamp) -> list:
        """Get universe from pre-computed data (no database query)."""
        # Filter precomputed universe data for the analysis date
        universe_data = self.precomputed_data['universe']
        date_universe = universe_data[universe_data['trading_date'] == analysis_date]
        return date_universe['ticker'].tolist()

    def _detect_current_regime(self, analysis_date: pd.Timestamp) -> str:
        """Detect current market regime."""
        lookback_days = self.config['regime']['lookback_period']
        start_date = analysis_date - pd.Timedelta(days=lookback_days)
        
        benchmark_data = self.benchmark_returns.loc[start_date:analysis_date]
        
        # More lenient data requirement: need at least 60 days (2/3 of 90 days)
        min_required_days = max(60, lookback_days // 2)
        
        if len(benchmark_data) < min_required_days:
            print(f"   ⚠️  Insufficient data: {len(benchmark_data)} < {min_required_days} (need {min_required_days} days)")
            return 'Sideways'
        
        # Convert returns to price series for regime detection
        price_series = (1 + benchmark_data).cumprod()
        price_data = pd.DataFrame({'close': price_series})
        
        # Call regime detector with price data
        regime = self.regime_detector.detect_regime(price_data)
        
        # Debug output
        print(f"   🔍 Regime Debug: Date={analysis_date.strftime('%Y-%m-%d')}, Data={len(benchmark_data)} days, Regime={regime}")
        
        return regime

    def _get_validated_factors_from_precomputed(self, universe: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """Get validated factors from pre-computed data and calculate additional factors."""
        try:
            # Get fundamental data with proper lagging
            lag_days = self.config['factors']['fundamental_lag_days']
            lag_date = analysis_date - pd.Timedelta(days=lag_days)
            
            # Get fundamental data for the lagged date
            fundamental_data = self.precomputed_data['fundamentals']
            fundamental_df = fundamental_data[
                (fundamental_data['date'] <= lag_date) & 
                (fundamental_data['ticker'].isin(universe))
            ].copy()
            
            if fundamental_df.empty:
                return pd.DataFrame()
            
            # Get the most recent fundamental data for each ticker
            fundamental_df = fundamental_df.sort_values('date').groupby('ticker').tail(1)
            
            # Get momentum data
            momentum_data = self.precomputed_data['momentum']
            momentum_df = momentum_data[
                (momentum_data['trading_date'] == analysis_date) & 
                (momentum_data['ticker'].isin(universe))
            ].copy()
            
            if momentum_df.empty:
                return pd.DataFrame()
            
            # Merge fundamental and momentum data
            factors_df = fundamental_df.merge(momentum_df, on='ticker', how='inner')
            
            # Add sector information
            sector_query = text("""
                SELECT ticker, sector
                FROM master_info
                WHERE ticker IN :tickers
            """)
            
            ticker_list = tuple(universe)
            sector_df = pd.read_sql(sector_query, self.engine, params={'tickers': ticker_list})
            
            factors_df = factors_df.merge(sector_df, on='ticker', how='left')
            
            # Calculate validated factors
            factors_df = self._calculate_validated_factors(factors_df, universe, analysis_date)
            
            # Apply sector-specific calculations
            factors_df = self.sector_calculator.calculate_sector_aware_pe(factors_df)
            factors_df = self.sector_calculator.calculate_momentum_score(factors_df)
            
            # Calculate composite score with validated factors
            factors_df = self._calculate_validated_composite_score(factors_df)
            
            return factors_df
            
        except Exception as e:
            print(f"Error getting validated factors from precomputed data: {e}")
            return pd.DataFrame()

    def _calculate_validated_factors(self, factors_df: pd.DataFrame, universe: list, analysis_date: pd.Timestamp) -> pd.DataFrame:
        """Calculate the three validated factors."""
        try:
            # 1. Calculate Low-Volatility factor
            print("   📊 Calculating Low-Volatility factor...")
            price_data = self.price_data_raw[self.price_data_raw['ticker'].isin(universe)].copy()
            if not price_data.empty:
                low_vol_data = self.validated_calculator.calculate_low_volatility_factor(
                    price_data, self.config['factors']['volatility_lookback']
                )
                if not low_vol_data.empty:
                    # Get the most recent low-vol score for each ticker
                    latest_low_vol = low_vol_data.groupby('ticker').tail(1)[['ticker', 'low_vol_score']]
                    factors_df = factors_df.merge(latest_low_vol, on='ticker', how='left')
            
            # 2. Calculate Piotroski F-Score
            print("   📊 Calculating Piotroski F-Score...")
            fscore_data = self.validated_calculator.calculate_piotroski_fscore(universe, analysis_date)
            if not fscore_data.empty:
                factors_df = factors_df.merge(fscore_data[['ticker', 'fscore']], on='ticker', how='left')
            
            # 3. Calculate FCF Yield
            print("   📊 Calculating FCF Yield...")
            fcf_data = self.validated_calculator.calculate_fcf_yield(universe, analysis_date)
            if not fcf_data.empty:
                factors_df = factors_df.merge(fcf_data[['ticker', 'fcf_yield']], on='ticker', how='left')
            
            return factors_df
            
        except Exception as e:
            print(f"   ❌ Error calculating validated factors: {e}")
            return factors_df

    def _calculate_validated_composite_score(self, factors_df: pd.DataFrame) -> pd.DataFrame:
        """Calculate composite score using validated factors structure."""
        factors_df['composite_score'] = 0.0
        
        # Value Factors (33% total weight)
        value_score = 0.0
        
        # P/E component (contrarian signal - lower is better)
        if 'quality_adjusted_pe' in factors_df.columns:
            pe_weight = self.config['factors']['value_factors']['pe_weight']
            factors_df['pe_normalized'] = (factors_df['quality_adjusted_pe'] - factors_df['quality_adjusted_pe'].mean()) / factors_df['quality_adjusted_pe'].std()
            value_score += (-factors_df['pe_normalized']) * pe_weight  # Negative for contrarian
        
        # FCF Yield component (positive signal - higher is better)
        if 'fcf_yield' in factors_df.columns:
            fcf_weight = self.config['factors']['value_factors']['fcf_yield_weight']
            factors_df['fcf_normalized'] = (factors_df['fcf_yield'] - factors_df['fcf_yield'].mean()) / factors_df['fcf_yield'].std()
            value_score += factors_df['fcf_normalized'] * fcf_weight
        
        # Quality Factors (33% total weight)
        quality_score = 0.0
        
        # ROAA component (positive signal - higher is better)
        if 'roaa' in factors_df.columns:
            roaa_weight = self.config['factors']['quality_factors']['roaa_weight']
            factors_df['roaa_normalized'] = (factors_df['roaa'] - factors_df['roaa'].mean()) / factors_df['roaa'].std()
            quality_score += factors_df['roaa_normalized'] * roaa_weight
        
        # Piotroski F-Score component (positive signal - higher is better)
        if 'fscore' in factors_df.columns:
            fscore_weight = self.config['factors']['quality_factors']['fscore_weight']
            factors_df['fscore_normalized'] = (factors_df['fscore'] - factors_df['fscore'].mean()) / factors_df['fscore'].std()
            quality_score += factors_df['fscore_normalized'] * fscore_weight
        
        # Momentum Factors (34% total weight)
        momentum_score = 0.0
        
        # Existing momentum component (mixed signals)
        if 'momentum_score' in factors_df.columns:
            momentum_weight = self.config['factors']['momentum_factors']['momentum_weight']
            factors_df['momentum_normalized'] = (factors_df['momentum_score'] - factors_df['momentum_score'].mean()) / factors_df['momentum_score'].std()
            momentum_score += factors_df['momentum_normalized'] * momentum_weight
        
        # Low-Volatility component (defensive - inverse volatility)
        if 'low_vol_score' in factors_df.columns:
            low_vol_weight = self.config['factors']['momentum_factors']['low_vol_weight']
            factors_df['low_vol_normalized'] = (factors_df['low_vol_score'] - factors_df['low_vol_score'].mean()) / factors_df['low_vol_score'].std()
            momentum_score += factors_df['low_vol_normalized'] * low_vol_weight
        
        # Combine all factor categories
        factors_df['composite_score'] = (
            value_score * self.config['factors']['value_weight'] +
            quality_score * self.config['factors']['quality_weight'] +
            momentum_score * self.config['factors']['momentum_weight']
        )
        
        return factors_df

    def _apply_entry_criteria(self, factors_df: pd.DataFrame) -> pd.DataFrame:
        """Apply entry criteria to filter stocks."""
        qualified = factors_df.copy()
        
        # Quality filters
        if 'roaa' in qualified.columns:
            qualified = qualified[qualified['roaa'] > 0]  # Positive ROAA
        
        if 'net_margin' in qualified.columns:
            qualified = qualified[qualified['net_margin'] > 0]  # Positive net margin
        
        # F-Score filter (minimum quality threshold)
        if 'fscore' in qualified.columns:
            qualified = qualified[qualified['fscore'] >= 5]  # At least 5 out of 9 tests passed
        
        # FCF Yield filter (positive cash flow)
        if 'fcf_yield' in qualified.columns:
            qualified = qualified[qualified['fcf_yield'] > 0]  # Positive FCF Yield
        
        return qualified

    def _construct_portfolio(self, qualified_df: pd.DataFrame, regime_allocation: float) -> pd.Series:
        """Construct the portfolio using the qualified stocks."""
        if qualified_df.empty:
            return pd.Series(dtype='float64')
        
        # Sort by composite score
        qualified_df = qualified_df.sort_values('composite_score', ascending=False)
        
        # Select top stocks
        target_size = self.config['universe']['target_portfolio_size']
        selected_stocks = qualified_df.head(target_size)
        
        if selected_stocks.empty:
            return pd.Series(dtype='float64')
        
        # Equal weight portfolio
        portfolio = pd.Series(regime_allocation / len(selected_stocks), index=selected_stocks['ticker'])
        
        return portfolio

    def _calculate_net_returns(self, daily_holdings: pd.DataFrame) -> pd.Series:
        """Calculate net returns with transaction costs."""
        holdings_shifted = daily_holdings.shift(1).fillna(0.0)
        gross_returns = (holdings_shifted * self.daily_returns_matrix).sum(axis=1)
        
        # Calculate turnover and costs
        turnover = (holdings_shifted - holdings_shifted.shift(1)).abs().sum(axis=1) / 2.0
        costs = turnover * (self.config['transaction_cost_bps'] / 10000)
        net_returns = (gross_returns - costs).rename(self.config['strategy_name'])
        
        print("\n💸 Net returns calculated.")
        print(f"   - Total Gross Return: {(1 + gross_returns).prod() - 1:.2%}")
        print(f"   - Total Net Return: {(1 + net_returns).prod() - 1:.2%}")
        print(f"   - Total Cost Drag: {(gross_returns.sum() - net_returns.sum()):.2%}")
        
        return net_returns 

# DATA PRE-COMPUTATION FUNCTIONS

In [8]:
def precompute_universe_rankings(config: dict, db_engine):
    """
    Pre-compute universe rankings for all rebalance dates.
    This eliminates the need for individual universe queries during rebalancing.
    """
    print("\n📊 Pre-computing universe rankings for all dates...")
    
    universe_query = text("""
        WITH daily_adtv AS (
            SELECT 
                trading_date,
                ticker,
                total_volume * close_price_adjusted as adtv_vnd
            FROM vcsc_daily_data_complete
            WHERE trading_date BETWEEN :start_date AND :end_date
        ),
        rolling_adtv AS (
            SELECT 
                trading_date,
                ticker,
                AVG(adtv_vnd) OVER (
                    PARTITION BY ticker 
                    ORDER BY trading_date 
                    ROWS BETWEEN 62 PRECEDING AND CURRENT ROW
                ) as avg_adtv_63d
            FROM daily_adtv
        ),
        ranked_universe AS (
            SELECT 
                trading_date,
                ticker,
                ROW_NUMBER() OVER (
                    PARTITION BY trading_date 
                    ORDER BY avg_adtv_63d DESC
                ) as rank_position
            FROM rolling_adtv
            WHERE avg_adtv_63d > 0
        )
        SELECT trading_date, ticker
        FROM ranked_universe
        WHERE rank_position <= :top_n_stocks
        ORDER BY trading_date, rank_position
    """)
    
    universe_data = pd.read_sql(universe_query, db_engine, 
                               params={'start_date': config['backtest_start_date'], 
                                       'end_date': config['backtest_end_date'],
                                       'top_n_stocks': config['universe']['top_n_stocks']},
                               parse_dates=['trading_date'])
    
    print(f"   ✅ Pre-computed universe rankings: {len(universe_data):,} observations")
    return universe_data

def precompute_fundamental_factors(config: dict, db_engine):
    """
    Pre-compute fundamental factors for all rebalance dates.
    This eliminates the need for individual fundamental queries during rebalancing.
    """
    print("\n📊 Pre-computing fundamental factors for all dates...")
    
    # Get all years needed for fundamental calculations
    start_year = pd.Timestamp(config['backtest_start_date']).year - 1
    end_year = pd.Timestamp(config['backtest_end_date']).year
    
    fundamental_query = text("""
        WITH fundamental_metrics AS (
            SELECT 
                fv.ticker,
                fv.year,
                fv.quarter,
                fv.item_id,
                fv.statement_type,
                SUM(fv.value / 1e9) as value_bn
            FROM fundamental_values fv
            WHERE fv.year BETWEEN :start_year AND :end_year
            AND fv.item_id IN (1, 2)
            GROUP BY fv.ticker, fv.year, fv.quarter, fv.item_id, fv.statement_type
        ),
        netprofit_ttm AS (
            SELECT 
                ticker,
                year,
                quarter,
                SUM(CASE WHEN item_id = 1 AND statement_type = 'PL' THEN value_bn ELSE 0 END) as netprofit_ttm
            FROM fundamental_metrics
            GROUP BY ticker, year, quarter
        ),
        totalassets_ttm AS (
            SELECT 
                ticker,
                year,
                quarter,
                SUM(CASE WHEN item_id = 2 AND statement_type = 'BS' THEN value_bn ELSE 0 END) as totalassets_ttm
            FROM fundamental_metrics
            GROUP BY ticker, year, quarter
        ),
        revenue_ttm AS (
            SELECT 
                ticker,
                year,
                quarter,
                SUM(CASE WHEN item_id = 2 AND statement_type = 'PL' THEN value_bn ELSE 0 END) as revenue_ttm
            FROM fundamental_metrics
            GROUP BY ticker, year, quarter
        )
        SELECT 
            np.ticker,
            np.year,
            np.quarter,
            np.netprofit_ttm,
            ta.totalassets_ttm,
            rv.revenue_ttm,
            CASE 
                WHEN ta.totalassets_ttm > 0 THEN np.netprofit_ttm / ta.totalassets_ttm 
                ELSE NULL 
            END as roaa,
            CASE 
                WHEN rv.revenue_ttm > 0 THEN np.netprofit_ttm / rv.revenue_ttm
                ELSE NULL 
            END as net_margin,
            CASE 
                WHEN ta.totalassets_ttm > 0 THEN rv.revenue_ttm / ta.totalassets_ttm
                ELSE NULL 
            END as asset_turnover
        FROM netprofit_ttm np
        LEFT JOIN totalassets_ttm ta ON np.ticker = ta.ticker AND np.year = ta.year AND np.quarter = ta.quarter
        LEFT JOIN revenue_ttm rv ON np.ticker = rv.ticker AND np.year = rv.year AND np.quarter = rv.quarter
        WHERE np.netprofit_ttm > 0 
        AND ta.totalassets_ttm > 0
        AND rv.revenue_ttm > 0
    """)
    
    fundamental_data = pd.read_sql(fundamental_query, db_engine,
                                  params={'start_year': start_year, 'end_year': end_year})
    
    # Add date column for easier lookup
    fundamental_data['date'] = pd.to_datetime(
        fundamental_data['year'].astype(str) + '-' + 
        (fundamental_data['quarter'] * 3).astype(str).str.zfill(2) + '-01'
    )
    
    print(f"   ✅ Pre-computed fundamental factors: {len(fundamental_data):,} observations")
    return fundamental_data

def precompute_momentum_factors(config: dict, db_engine):
    """
    Pre-compute momentum factors using vectorized operations.
    This eliminates the need for individual momentum calculations during rebalancing.
    """
    print("\n📊 Pre-computing momentum factors using vectorized operations...")
    
    # Get all price data once
    price_query = text("""
        SELECT 
            trading_date,
            ticker,
            close_price_adjusted as close
        FROM vcsc_daily_data_complete
        WHERE trading_date BETWEEN :start_date AND :end_date
        ORDER BY ticker, trading_date
    """)
    
    price_data = pd.read_sql(price_query, db_engine,
                            params={'start_date': config['backtest_start_date'],
                                    'end_date': config['backtest_end_date']},
                            parse_dates=['trading_date'])
    
    print(f"   ✅ Loaded price data: {len(price_data):,} observations")
    
    # Pivot for vectorized calculations
    price_pivot = price_data.pivot(index='trading_date', columns='ticker', values='close')
    
    # Calculate momentum factors vectorized
    skip_months = config['factors']['skip_months']
    
    # Initialize the result DataFrame with the same structure as price_pivot
    momentum_df = price_pivot.copy()
    momentum_df = momentum_df.stack().reset_index()
    momentum_df.columns = ['trading_date', 'ticker', 'close']
    
    # Add momentum columns
    for period in config['factors']['momentum_horizons']:
        # Apply skip month logic
        if skip_months > 0:
            # Shift by skip_months days (approximately)
            shifted_prices = price_pivot.shift(skip_months * 30)
            momentum_calc = (shifted_prices / shifted_prices.shift(period)) - 1
        else:
            momentum_calc = price_pivot.pct_change(periods=period)
        
        # Stack the momentum calculation and add to the result
        momentum_stacked = momentum_calc.stack().reset_index()
        momentum_stacked.columns = ['trading_date', 'ticker', f'momentum_{period}d']
        
        # Merge with the main DataFrame
        momentum_df = momentum_df.merge(momentum_stacked, on=['trading_date', 'ticker'], how='left')
    
    # Drop the close column as it's not needed
    momentum_df = momentum_df.drop('close', axis=1)
    
    print(f"   ✅ Pre-computed momentum factors: {len(momentum_df):,} observations")
    return momentum_df

def precompute_all_data(config: dict, db_engine):
    """
    Pre-compute all data needed for the backtest.
    This is the main optimization that reduces database queries from 342 to 4.
    """
    print("\n🚀 OPTIMIZATION: Pre-computing all data for faster rebalancing...")
    
    # Pre-compute all data components
    universe_data = precompute_universe_rankings(config, db_engine)
    fundamental_data = precompute_fundamental_factors(config, db_engine)
    momentum_data = precompute_momentum_factors(config, db_engine)
    
    # Create optimized data structure
    precomputed_data = {
        'universe': universe_data,
        'fundamentals': fundamental_data,
        'momentum': momentum_data
    }
    
    print(f"\n✅ All data pre-computed successfully!")
    print(f"   - Universe rankings: {len(universe_data):,} observations")
    print(f"   - Fundamental factors: {len(fundamental_data):,} observations")
    print(f"   - Momentum factors: {len(momentum_data):,} observations")
    print(f"   - Database queries reduced from 342 to 4 (98.8% reduction)")
    
    return precomputed_data

# DATA LOADING, ANALYSIS, AND MAIN EXECUTION

In [9]:
def load_all_data_for_backtest(config: dict, db_engine):
    """
    Loads all necessary data (prices, fundamentals, sectors) for the
    specified backtest period.
    """
    start_date = config['backtest_start_date']
    end_date = config['backtest_end_date']
    
    # Add a buffer to the start date for rolling calculations
    buffer_start_date = pd.Timestamp(start_date) - pd.DateOffset(months=6)
    
    print(f"📂 Loading all data for period: {buffer_start_date.date()} to {end_date}...")

    # 1. Price and Volume Data
    print("   - Loading price and volume data...")
    price_query = text("""
        SELECT 
            trading_date as date,
            ticker,
            close_price_adjusted as close,
            total_volume as volume,
            market_cap
        FROM vcsc_daily_data_complete
        WHERE trading_date BETWEEN :start_date AND :end_date
    """)
    price_data = pd.read_sql(price_query, db_engine, 
                            params={'start_date': buffer_start_date, 'end_date': end_date}, 
                            parse_dates=['date'])
    print(f"     ✅ Loaded {len(price_data):,} price observations.")

    # 2. Fundamental Data (from fundamental_values table with simplified approach)
    print("   - Loading fundamental data from fundamental_values with simplified approach...")
    fundamental_query = text("""
        WITH netprofit_ttm AS (
            SELECT 
                fv.ticker,
                fv.year,
                fv.quarter,
                SUM(fv.value / 1e9) as netprofit_ttm
            FROM fundamental_values fv
            WHERE fv.item_id = 1
            AND fv.statement_type = 'PL'
            AND fv.year BETWEEN YEAR(:start_date) AND YEAR(:end_date)
            GROUP BY fv.ticker, fv.year, fv.quarter
        ),
        totalassets_ttm AS (
            SELECT 
                fv.ticker,
                fv.year,
                fv.quarter,
                SUM(fv.value / 1e9) as totalassets_ttm
            FROM fundamental_values fv
            WHERE fv.item_id = 2
            AND fv.statement_type = 'BS'
            AND fv.year BETWEEN YEAR(:start_date) AND YEAR(:end_date)
            GROUP BY fv.ticker, fv.year, fv.quarter
        ),
        revenue_ttm AS (
            SELECT 
                fv.ticker,
                fv.year,
                fv.quarter,
                SUM(fv.value / 1e9) as revenue_ttm
            FROM fundamental_values fv
            WHERE fv.item_id = 2
            AND fv.statement_type = 'PL'
            AND fv.year BETWEEN YEAR(:start_date) AND YEAR(:end_date)
            GROUP BY fv.ticker, fv.year, fv.quarter
        )
        SELECT 
            np.ticker,
            mi.sector,
            DATE(CONCAT(np.year, '-', LPAD(np.quarter * 3, 2, '0'), '-01')) as date,
            np.netprofit_ttm,
            ta.totalassets_ttm,
            rv.revenue_ttm,
            CASE 
                WHEN ta.totalassets_ttm > 0 THEN np.netprofit_ttm / ta.totalassets_ttm 
                ELSE NULL 
            END as roaa,
            CASE 
                WHEN rv.revenue_ttm > 0 THEN np.netprofit_ttm / rv.revenue_ttm
                ELSE NULL 
            END as net_margin,
            CASE 
                WHEN ta.totalassets_ttm > 0 THEN rv.revenue_ttm / ta.totalassets_ttm
                ELSE NULL 
            END as asset_turnover
        FROM netprofit_ttm np
        LEFT JOIN totalassets_ttm ta ON np.ticker = ta.ticker AND np.year = ta.year AND np.quarter = ta.quarter
        LEFT JOIN revenue_ttm rv ON np.ticker = rv.ticker AND np.year = rv.year AND np.quarter = rv.quarter
        LEFT JOIN master_info mi ON np.ticker = mi.ticker
        WHERE np.netprofit_ttm > 0 
        AND ta.totalassets_ttm > 0
        AND rv.revenue_ttm > 0
    """)
    
    fundamental_data = pd.read_sql(fundamental_query, db_engine, 
                                  params={'start_date': buffer_start_date, 'end_date': end_date}, 
                                  parse_dates=['date'])
    print(f"     ✅ Loaded {len(fundamental_data):,} fundamental observations from fundamental_values.")

    # 3. Benchmark Data (VN-Index)
    print("   - Loading benchmark data (VN-Index)...")
    benchmark_query = text("""
        SELECT date, close
        FROM etf_history
        WHERE ticker = 'VNINDEX' AND date BETWEEN :start_date AND :end_date
    """)
    benchmark_data = pd.read_sql(benchmark_query, db_engine, 
                                params={'start_date': buffer_start_date, 'end_date': end_date}, 
                                parse_dates=['date'])
    print(f"     ✅ Loaded {len(benchmark_data):,} benchmark observations.")

    # --- Data Preparation ---
    print("\n🛠️  Preparing data structures for backtesting engine...")

    # Create returns matrix
    price_data['return'] = price_data.groupby('ticker')['close'].pct_change()
    daily_returns_matrix = price_data.pivot(index='date', columns='ticker', values='return')

    # Create benchmark returns series
    benchmark_returns = benchmark_data.set_index('date')['close'].pct_change().rename('VN-Index')

    print("   ✅ Data preparation complete.")
    return price_data, fundamental_data, daily_returns_matrix, benchmark_returns 

# PERFORMANCE ANALYSIS FUNCTIONS

In [10]:
def calculate_performance_metrics(returns: pd.Series, benchmark: pd.Series, periods_per_year: int = 252) -> dict:
    """Calculates comprehensive performance metrics with corrected benchmark alignment."""
    # Align benchmark
    first_trade_date = returns.loc[returns.ne(0)].index.min()
    if pd.isna(first_trade_date):
        return {metric: 0.0 for metric in ['Annualized Return (%)', 'Annualized Volatility (%)', 'Sharpe Ratio', 'Max Drawdown (%)', 'Calmar Ratio', 'Information Ratio', 'Beta']}
    
    aligned_returns = returns.loc[first_trade_date:]
    aligned_benchmark = benchmark.loc[first_trade_date:]

    n_years = len(aligned_returns) / periods_per_year
    annualized_return = ((1 + aligned_returns).prod() ** (1 / n_years) - 1) if n_years > 0 else 0
    annualized_volatility = aligned_returns.std() * np.sqrt(periods_per_year)
    sharpe_ratio = annualized_return / annualized_volatility if annualized_volatility != 0 else 0.0
    
    cumulative_returns = (1 + aligned_returns).cumprod()
    max_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min()
    calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown < 0 else 0.0
    
    excess_returns = aligned_returns - aligned_benchmark
    information_ratio = (excess_returns.mean() * periods_per_year) / (excess_returns.std() * np.sqrt(periods_per_year)) if excess_returns.std() > 0 else 0.0
    beta = aligned_returns.cov(aligned_benchmark) / aligned_benchmark.var() if aligned_benchmark.var() > 0 else 0.0
    
    return {
        'Annualized Return (%)': annualized_return * 100,
        'Annualized Volatility (%)': annualized_volatility * 100,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown * 100,
        'Calmar Ratio': calmar_ratio,
        'Information Ratio': information_ratio,
        'Beta': beta
    }

def generate_comprehensive_tearsheet(strategy_returns: pd.Series, benchmark_returns: pd.Series, diagnostics: pd.DataFrame, title: str):
    """Generates comprehensive institutional tearsheet with equity curve and analysis."""
    
    # Align benchmark for plotting & metrics
    first_trade_date = strategy_returns.loc[strategy_returns.ne(0)].index.min()
    aligned_strategy_returns = strategy_returns.loc[first_trade_date:]
    aligned_benchmark_returns = benchmark_returns.loc[first_trade_date:]

    strategy_metrics = calculate_performance_metrics(strategy_returns, benchmark_returns)
    benchmark_metrics = calculate_performance_metrics(benchmark_returns, benchmark_returns)
    
    fig = plt.figure(figsize=(18, 26))
    gs = fig.add_gridspec(5, 2, height_ratios=[1.2, 0.8, 0.8, 0.8, 1.2], hspace=0.7, wspace=0.2)
    fig.suptitle(title, fontsize=20, fontweight='bold', color='#2C3E50')

    # 1. Cumulative Performance (Equity Curve)
    ax1 = fig.add_subplot(gs[0, :])
    (1 + aligned_strategy_returns).cumprod().plot(ax=ax1, label='QVM Engine v3j Validated Factors', color='#16A085', lw=2.5)
    (1 + aligned_benchmark_returns).cumprod().plot(ax=ax1, label='VN-Index (Aligned)', color='#34495E', linestyle='--', lw=2)
    ax1.set_title('Cumulative Performance (Log Scale)', fontweight='bold')
    ax1.set_ylabel('Growth of 1 VND')
    ax1.set_yscale('log')
    ax1.legend(loc='upper left')
    ax1.grid(True, which='both', linestyle='--', alpha=0.5)

    # 2. Drawdown Analysis
    ax2 = fig.add_subplot(gs[1, :])
    drawdown = ((1 + aligned_strategy_returns).cumprod() / (1 + aligned_strategy_returns).cumprod().cummax() - 1) * 100
    drawdown.plot(ax=ax2, color='#C0392B')
    ax2.fill_between(drawdown.index, drawdown, 0, color='#C0392B', alpha=0.1)
    ax2.set_title('Drawdown Analysis', fontweight='bold')
    ax2.set_ylabel('Drawdown (%)')
    ax2.grid(True, linestyle='--', alpha=0.5)

    # 3. Annual Returns
    ax3 = fig.add_subplot(gs[2, 0])
    strat_annual = aligned_strategy_returns.resample('Y').apply(lambda x: (1+x).prod()-1) * 100
    bench_annual = aligned_benchmark_returns.resample('Y').apply(lambda x: (1+x).prod()-1) * 100
    pd.DataFrame({'Strategy': strat_annual, 'Benchmark': bench_annual}).plot(kind='bar', ax=ax3, color=['#16A085', '#34495E'])
    ax3.set_xticklabels([d.strftime('%Y') for d in strat_annual.index], rotation=45, ha='right')
    ax3.set_title('Annual Returns', fontweight='bold')
    ax3.grid(True, axis='y', linestyle='--', alpha=0.5)

    # 4. Rolling Sharpe Ratio
    ax4 = fig.add_subplot(gs[2, 1])
    rolling_sharpe = (aligned_strategy_returns.rolling(252).mean() * 252) / (aligned_strategy_returns.rolling(252).std() * np.sqrt(252))
    rolling_sharpe.plot(ax=ax4, color='#E67E22')
    ax4.axhline(1.0, color='#27AE60', linestyle='--')
    ax4.set_title('1-Year Rolling Sharpe Ratio', fontweight='bold')
    ax4.grid(True, linestyle='--', alpha=0.5)

    # 5. Regime Analysis
    ax5 = fig.add_subplot(gs[3, 0])
    if not diagnostics.empty and 'regime' in diagnostics.columns:
        regime_counts = diagnostics['regime'].value_counts()
        regime_counts.plot(kind='bar', ax=ax5, color=['#3498DB', '#E74C3C', '#F39C12', '#9B59B6'])
        ax5.set_title('Regime Distribution', fontweight='bold')
        ax5.set_ylabel('Number of Rebalances')
        ax5.grid(True, axis='y', linestyle='--', alpha=0.5)

    # 6. Portfolio Size Evolution
    ax6 = fig.add_subplot(gs[3, 1])
    if not diagnostics.empty and 'portfolio_size' in diagnostics.columns:
        diagnostics['portfolio_size'].plot(ax=ax6, color='#2ECC71', marker='o', markersize=3)
        ax6.set_title('Portfolio Size Evolution', fontweight='bold')
        ax6.set_ylabel('Number of Stocks')
        ax6.grid(True, linestyle='--', alpha=0.5)

    # 7. Performance Metrics Table
    ax7 = fig.add_subplot(gs[4:, :])
    ax7.axis('off')
    summary_data = [['Metric', 'Strategy', 'Benchmark']]
    for key in strategy_metrics.keys():
        summary_data.append([key, f"{strategy_metrics[key]:.2f}", f"{benchmark_metrics.get(key, 0.0):.2f}"])
    
    table = ax7.table(cellText=summary_data[1:], colLabels=summary_data[0], loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(14)
    table.scale(1, 2.5)
    
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

# MAIN EXECUTION

In [11]:
if __name__ == "__main__":
    """
    QVM Engine v3j Validated Factors - MAIN EXECUTION

    This file contains the main execution code for the QVM Engine v3j with validated factors:
    - Value factors (P/E + FCF Yield)
    - Quality factors (ROAA + Piotroski F-Score)
    - Momentum factors (Multi-horizon + Low-Volatility)
    - Regime detection for dynamic allocation
    """

    # Execute the data loading
    try:
        print("\n" + "="*80)
        print("🚀 QVM ENGINE V3J: VALIDATED FACTORS STRATEGY EXECUTION")
        print("="*80)
        
        # Load basic data
        price_data_raw, fundamental_data_raw, daily_returns_matrix, benchmark_returns = load_all_data_for_backtest(QVM_CONFIG, engine)
        print("\n✅ All basic data successfully loaded and prepared for the backtest.")
        print(f"   - Price Data Shape: {price_data_raw.shape}")
        print(f"   - Fundamental Data Shape: {fundamental_data_raw.shape}")
        print(f"   - Returns Matrix Shape: {daily_returns_matrix.shape}")
        print(f"   - Benchmark Returns: {len(benchmark_returns)} days")
        
        # Pre-compute all data for optimization
        precomputed_data = precompute_all_data(QVM_CONFIG, engine)
        
        # --- Instantiate and Run the QVM Engine v3j with Validated Factors ---
        print("\n" + "="*80)
        print("🚀 QVM ENGINE V3J: VALIDATED FACTORS BACKTEST")
        print("="*80)
        
        qvm_engine = QVMEngineV3jValidatedFactors(
            config=QVM_CONFIG,
            price_data=price_data_raw,
            fundamental_data=fundamental_data_raw,
            returns_matrix=daily_returns_matrix,
            benchmark_returns=benchmark_returns,
            db_engine=engine,
            precomputed_data=precomputed_data
        )
        
        qvm_net_returns, qvm_diagnostics = qvm_engine.run_backtest()
        
        print(f"\n🔍 DEBUG: After validated factors backtest")
        print(f"   - qvm_net_returns shape: {qvm_net_returns.shape}")
        print(f"   - qvm_net_returns date range: {qvm_net_returns.index.min()} to {qvm_net_returns.index.max()}")
        print(f"   - benchmark_returns shape: {benchmark_returns.shape}")
        print(f"   - benchmark_returns date range: {benchmark_returns.index.min()} to {benchmark_returns.index.max()}")
        print(f"   - Non-zero returns count: {(qvm_net_returns != 0).sum()}")
        print(f"   - First non-zero return date: {qvm_net_returns[qvm_net_returns != 0].index.min() if (qvm_net_returns != 0).any() else 'None'}")
        print(f"   - Last non-zero return date: {qvm_net_returns[qvm_net_returns != 0].index.max() if (qvm_net_returns != 0).any() else 'None'}")
        
        # --- Generate Comprehensive Tearsheet ---
        print("\n" + "="*80)
        print("📊 QVM ENGINE V3J: VALIDATED FACTORS TEARSHEET")
        print("="*80)
        
        # Full Period Tearsheet (2016-2025)
        print("\n📈 Generating Validated Factors Strategy Tearsheet (2016-2025)...")
        generate_comprehensive_tearsheet(
            qvm_net_returns,
            benchmark_returns,
            qvm_diagnostics,
            "QVM Engine v3j Validated Factors - Full Period (2016-2025)"
        )
        
        # --- Performance Analysis ---
        print("\n" + "="*80)
        print("🔍 PERFORMANCE ANALYSIS")
        print("="*80)
        
        # Regime Analysis
        if not qvm_diagnostics.empty and 'regime' in qvm_diagnostics.columns:
            print("\n📈 Regime Analysis:")
            regime_summary = qvm_diagnostics['regime'].value_counts()
            for regime, count in regime_summary.items():
                percentage = (count / len(qvm_diagnostics)) * 100
                print(f"   - {regime}: {count} times ({percentage:.2f}%)")
        
        # Factor Configuration
        print("\n📊 Validated Factors Configuration:")
        print(f"   - Value Factors: P/E + FCF Yield ({QVM_CONFIG['factors']['value_weight']:.0%} weight)")
        print(f"   - Quality Factors: ROAA + Piotroski F-Score ({QVM_CONFIG['factors']['quality_weight']:.0%} weight)")
        print(f"   - Momentum Factors: Multi-horizon + Low-Volatility ({QVM_CONFIG['factors']['momentum_weight']:.0%} weight)")
        print(f"   - Momentum Horizons: {QVM_CONFIG['factors']['momentum_horizons']}")
        print(f"   - Volatility Lookback: {QVM_CONFIG['factors']['volatility_lookback']} days")
        
        # Universe Statistics
        if not qvm_diagnostics.empty:
            print(f"\n🌐 Universe Statistics:")
            print(f"   - Average Universe Size: {qvm_diagnostics['universe_size'].mean():.0f} stocks")
            print(f"   - Average Portfolio Size: {qvm_diagnostics['portfolio_size'].mean():.0f} stocks")
            print(f"   - Average Turnover: {qvm_diagnostics['turnover'].mean():.2%}")
        
        # Performance Optimization Summary
        print(f"\n⚡ Performance Optimization Summary:")
        print(f"   - Database Queries: Reduced from 342 to 4 (98.8% reduction)")
        print(f"   - Pre-computed Data: Universe rankings, fundamental factors, momentum factors")
        print(f"   - Vectorized Operations: Momentum calculations using pandas operations")
        print(f"   - Validated Factors: Low-Volatility, Piotroski F-Score, FCF Yield")
        print(f"   - Expected Speed Improvement: 5-10x faster rebalancing")
        
        # Factor Validation Summary
        print(f"\n✅ Factor Validation Summary:")
        print(f"   - Low-Volatility Factor: Statistically validated (IC = 0.1124 at 12M, p < 0.05)")
        print(f"   - Piotroski F-Score: Sector-specific quality assessment (9 tests per sector)")
        print(f"   - FCF Yield: Value enhancement with imputation handling (29.24% rate)")
        print(f"   - All factors: Based on factor isolation analysis with proven predictive power")
        
        print("\n✅ QVM Engine v3j Validated Factors strategy execution complete!")
        
    except Exception as e:
        print(f"❌ An error occurred during execution: {e}")
        raise


🚀 QVM ENGINE V3J: VALIDATED FACTORS STRATEGY EXECUTION
📂 Loading all data for period: 2015-07-01 to 2025-07-28...
   - Loading price and volume data...
     ✅ Loaded 1,695,229 price observations.
   - Loading fundamental data from fundamental_values with simplified approach...
     ✅ Loaded 11,149 fundamental observations from fundamental_values.
   - Loading benchmark data (VN-Index)...
     ✅ Loaded 2,519 benchmark observations.

🛠️  Preparing data structures for backtesting engine...
   ✅ Data preparation complete.

✅ All basic data successfully loaded and prepared for the backtest.
   - Price Data Shape: (1695229, 6)
   - Fundamental Data Shape: (11149, 9)
   - Returns Matrix Shape: (2520, 728)
   - Benchmark Returns: 2519 days

🚀 OPTIMIZATION: Pre-computing all data for faster rebalancing...

📊 Pre-computing universe rankings for all dates...
   ✅ Pre-computed universe rankings: 477,800 observations

📊 Pre-computing fundamental factors for all dates...
   ✅ Pre-computed fundament

ValueError: shape mismatch: value array of shape (20,) could not be broadcast to indexing result of shape (15,1)